This notebook computes the derivative of the resilience indicator. This information is usefull for rendering the scorcards. This notebook must be run after compute_res_ind and before render_scorecards

In [1]:
%reset-f
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from res_ind_lib import *

# data

In [2]:
#Load the excel data file into a dataframe (table)
df_original=pd.read_excel("inputs/all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the first column (Provinces) as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variables and the 3rd (2) row is empty. We skip them
                ) 
df_original.index.name="province"
#Displays the first rows of that dataframe
df_original=def_ref_values(df_original)

# computes all derivatives

In [3]:
deriv_set = np.setdiff1d( df_original.columns,
    ["pop","iso3","original_country","income_elast","avg_prod_k","axfin_p","axfin_r","v_s","gdp_pc_pp","gdp_pc_pp_nat",
     "protectionref","gdp_pc_pp_ref","rho" ])

pd.DataFrame(data=deriv_set).to_csv("inputs/deriv_set.csv",index=False,header=False)

pol_assess_set = ["dKtot","dWtot_currency"]
              
#Checks that info has information for all variables in deriv_set
info = pd.read_csv("inputs/inputs_info.csv").set_index("key")
for d in deriv_set:
    if d not in info.index.values:
        raise Exception(d+" is not documented in inputs_info.csv")

        


In [4]:
from progress_reporter import *

def compute_derivative(df_original,deriv_set,pol_assess_set):
    der = pd.DataFrame(index=df_original.index, columns=pd.MultiIndex.from_product([deriv_set,pol_assess_set], names=['inputs', 'outputs']))
    h=0.0001
    #loop on all data in df prior to add the results
    fx = compute_resiliences(df_original)[pol_assess_set]
    for var in deriv_set:
        try:
            progress_reporter(var)
            df_=df_original.copy(deep=True)
            df_[var]=df_[var]+h
            fxh= compute_resiliences(df_)[pol_assess_set]
            der[var] = ((fxh-fx)/(h)).mul(df_[var],axis=0)/fx
        except TypeError:
            print("no derivative for " +var)
    progress_reporter("done.")        
    return der.swaplevel('inputs', 'outputs', axis=1).sort_index(axis=1)

In [5]:
derivatives = compute_derivative(df_original,deriv_set,pol_assess_set)

Currently working on:  done.


# output

output signs in excel

In [6]:
#saves derivatives in excel tabs with signs in colors (very usefull for understanding the model)
writer= pd.ExcelWriter("results/derivatives.xlsx", engine='xlsxwriter')
workbook=writer.book
# Add a format. Light red fill with dark red text.
red = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

blue = workbook.add_format({'bg_color': '#92c5de',
                               'font_color': '#000061'})
for outname in pol_assess_set:
    sheetname = outname[0:30]
    #for outname in ["resilience"]:
    (derivatives[outname].dropna()).to_excel(writer,sheet_name=sheetname)
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '>',
                                    'value':    0,
                                    'format':   blue})
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '<',
                                    'value':    0,
                                    'format':   red})
    writer.sheets[sheetname].freeze_panes(1, 1)


In [7]:
try :
    writer.save()
except PermissionError:
    warnings.warn("Cannot write excel file. Check that it's not opened and try again")


# Double checks the results

In [8]:
#Signs of resilience derivative
for out in pol_assess_set:
    der =     np.sign(derivatives[out])
    signs= pd.Series(index=der.columns)
    for i in signs.index:
        if (der[i].min()==der[i].max()): #all nonnan signs are equal
            signs[i]=der[i].min()
        else:
            print("ambigous sign for "+i+" on "+out)
            signs[i]=np.nan

